In [ ]:
baseUrl = "https://apl.unob.cz/StudijniProgramy/Studium/4/Cyber-Security?culture=cs-CZ"

https://docs.aiohttp.org/en/stable/client_quickstart.html

In [ ]:
import aiohttp
import asyncio

In [ ]:
async with aiohttp.ClientSession() as session:
    async with session.get(baseUrl) as resp:
        print(resp.status)
        htmlData = (await resp.text()) #ulozeni html

In [ ]:
#prace s html:
position = htmlData.find('<table') #hledam kde je v textu <table = zacatek tabulky
positionEnd = htmlData.find('</table') #konec tabulky

print(position, positionEnd)

In [ ]:
tableData = htmlData[position:positionEnd]
rows = tableData.split('<tr')
for row in rows:
    print(row.replace('\n',''))

chceme vědět, když se určitá fce někde v kodu zavola

pracujeme s daty z internetu, není žádoucí data stahovat opakovaně. V případě synchronní bychom použili dekorátor @cache, pro asynchronní si vyvorříme fci sami
tzv. dekorování:

In [ ]:
#ziskavani html stranek optimalizuje, nemusi se stahovat znovu a znovu
async def singleCall(asyncFunc): #vezme funkci a nahradí ji jinou funkcí
    awaitedResults = {}
    async def resultFunc(singleParam):
        result = awaitedResults.get(singleParam, None)
        if result is None: #zname vysledek? ne:
            result = await asyncFunc(singleParam) #ziskame ho
            awaitedResults[singleParam] = result #ulozime ho
        return result
    return resultFunc

In [ ]:
import aiohttp
baseUrl = "https://apl.unob.cz/StudijniProgramy/Studium/4/Kyberneticka-bezpecnost"

async def getData(baseUrl):
    async with aiohttp.ClientSession() as session:
        async with session.get(baseUrl) as resp:
            htmlData = (await resp.text()) #ulozeni html
    return htmlData

V získané stránce si najdeme všechny výskyty vzoru. Všimněte si strukuty na výstupu (tuple)

In [ ]:
import re
textData = await getData(baseUrl)
#print(textData[5000:6000])
processor = re.compile('(\/StudijniProgramy/PredmetDetail\/([0-9]+)\/([^\"]+))') #najdi mi v textu vyraz, ktery je v zavorce
# [0-9]+ ... libovolne cislo s libovolnym poctem opakeovani
# \      ... krome
# [^\"]  ... 
items = processor.findall(textData)
for item in items:
    print(item)

pro lepší použití vyvtoříme funkci:

In [ ]:
def extractData(data, pattern):
    processor = re.compile('(\/StudijniProgramy/PredmetDetail\/([0-9]+)\/([^\"]+))')
    items = processor.findall(textData)
    for item in items:
        yield item

jednotlive prvky tuple pojmenujeme, chceme json

In [ ]:
def nameTuple(data, names): #spoji jmeno a data v jednom radku
    result = {}
    for key, value in zip(names, data):
        result[key] = value
    return result

def nameAllRows(data, names):
    for row in data:
        yield nameTuple(row, names)

baseUrl = "https://apl.unob.cz/StudijniProgramy/Studium/4/Kyberneticka-bezpecnost"
pattern = '(\/StudijniProgramy/PredmetDetail\/([0-9]+)\/([^\"]+))'
names = ["url","id","name"]

data = await getData(baseUrl)
tuples = extractData(data, pattern)
jsons = nameAllRows(data, names)

for item in jsons:
    print(item)
    